## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-05-10-49-00 +0000,wsj,Energy & Utilities Roundup: Market Talk,https://www.wsj.com/business/energy-utilities-...
1,2025-11-05-10-39-00 +0000,wsj,Volatile Risk Sentiment Could Limit Euro’s Rise,https://www.wsj.com/finance/currencies/asian-c...
2,2025-11-05-10-36-54 +0000,bbc,Rama Duwaji: Who is the wife of NYC's mayor-el...,https://www.bbc.com/news/articles/cx2lg3z3z4zo...
3,2025-11-05-10-36-28 +0000,bbc,Mortgages and AI to be added to the curriculum...,https://www.bbc.com/news/articles/cddr3v6j9mmo...
4,2025-11-05-10-34-44 +0000,wapo,"A Muslim mayor, a female governor and other no...",https://www.washingtonpost.com/politics/2025/1...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2324/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
149,trump,44
174,new,34
18,mamdani,27
30,election,22
15,mayor,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
88,2025-11-05-06-34-00 +0000,wsj,The Democratic Party won the New York City may...,https://www.wsj.com/politics/elections/five-ta...,227
119,2025-11-05-04-42-00 +0000,wsj,"Andrew Cuomo, part of New York’s political fir...",https://www.wsj.com/politics/elections/half-a-...,165
116,2025-11-05-04-58-03 +0000,bbc,'Mandate for change': Zohran Mamdani wins New ...,https://www.bbc.com/news/articles/ceq01l8reqlo...,145
286,2025-11-04-21-01-07 +0000,nyt,"In New Jersey, Democrats and Republicans say T...",https://www.nytimes.com/live/2025/11/04/us/ele...,141
79,2025-11-05-07-26-00 +0000,wsj,Zohran Mamdani sealed his mayoral victory in N...,https://www.wsj.com/politics/elections/the-vot...,139


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
88,227,2025-11-05-06-34-00 +0000,wsj,The Democratic Party won the New York City may...,https://www.wsj.com/politics/elections/five-ta...
119,120,2025-11-05-04-42-00 +0000,wsj,"Andrew Cuomo, part of New York’s political fir...",https://www.wsj.com/politics/elections/half-a-...
75,76,2025-11-05-07-34-00 +0000,wsj,Lawmakers are hopeful they can reach a biparti...,https://www.wsj.com/politics/policy/lawmakers-...
165,65,2025-11-05-02-17-39 +0000,nypost,Virginia Governor-elect Abigail Spanberger’ ca...,https://nypost.com/2025/11/04/us-news/virginia...
303,52,2025-11-04-19-43-00 +0000,wsj,The Trump administration will comply with a co...,https://www.wsj.com/politics/policy/trump-thre...
308,49,2025-11-04-19-21-43 +0000,nypost,"Trump White House snubs ex-VP Dick Cheney, off...",https://nypost.com/2025/11/04/us-news/trump-wh...
297,48,2025-11-04-20-01-11 +0000,cbc,How budget day unfolded in Ottawa — from budge...,https://www.cbc.ca/news/politics/livestory/how...
80,48,2025-11-05-07-20-00 +0000,wsj,UPS Cargo Plane Crash in Kentucky Leaves at Le...,https://www.wsj.com/us-news/ups-cargo-plane-cr...
244,41,2025-11-04-22-32-19 +0000,wapo,AI stocks waver as ‘Big Short’ investor bets a...,https://www.washingtonpost.com/business/2025/1...
192,40,2025-11-05-00-38-00 +0000,wsj,President Trump renominated Jared Isaacman to ...,https://www.wsj.com/politics/elections/trump-r...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
